<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [45]:
import pandas as pd
import tweepy

In [46]:
# Leer las llaves
keys = pd.read_csv("C:/Users/CHOMELY/Documents/1. Especialización Konrad Lorenz/1. Procesamiento de Lenguaje Natural (NLP)/misllaves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [47]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True) #decimo a twitter quienes somos wait on rete limite, permtie que twitter no bloquee cuando se hacen muchas peticiones

In [48]:
tweets = tweepy.Cursor(api.search,
                       q=['ESMAD, POLICIA -RT'], #Lista de términos que se quieren buscar, no importa si es en mayus o minusc -RT quite los tuits que son retuiteados es decir duplicados
                       lang='es', #Español
                       geocode='3.42158,-76.5205,100km', #coordenadas de cali , cadena de texto indicar un punto y buscar los tuits en ese punto occidente es negativo norte es positiva sur negativo y km a la redonda
                       tweet_mode='extended', #Para que traiga más de 140 caracteres 280 cacteres ahora entonces el parametro es para evitar que no llegeun truncados
                       include_rts=False, #tuits retuiteados evitando lo que han sido retuiteados
                       ).items(500)

In [49]:
# Objeto generador
t = [tweet for tweet in tweets]

In [50]:
t[0].created_at

datetime.datetime(2021, 5, 20, 6, 18, 58)

In [51]:
t[0].user.location

'Colombia'

In [52]:
t[0].full_text

'Duque si hizo algo por Colombia: \n\nDuque hundió al Centro Democrático\nDuque enterró a Uribe\nDuque nos abrió los ojos \nDuque satanizó al esmad y a la policia\nDuque glorificó a la primera linea\nDuque unió a las barras bravas\nDuque puso los ojos del mundo sobre Uribe\n#PrimeraLinea'

In [53]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Cali')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)

In [54]:
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweets,date,city,user_location,id,link
0,Duque si hizo algo por Colombia: \n\nDuque hun...,2021-05-20 06:18:58,Cali,Colombia,1395262756799004672,https://twitter.com/i/web/stuatus/139526275679...
1,@GobiernoBTA @andresidarragaf Que horror! No s...,2021-05-20 06:08:06,Cali,"Bogotá, D.C., Colombia",1395260023643705348,https://twitter.com/i/web/stuatus/139526002364...
2,Hoy en la multitudinaria movilización en Cali ...,2021-05-20 06:02:42,Cali,"Cali, Valle del Cauca",1395258664051691522,https://twitter.com/i/web/stuatus/139525866405...
3,@VickyDavilaH @AlvaroUribeVel @IvanDuque esto ...,2021-05-20 06:01:12,Cali,"Pradera, Colombia",1395258285477990400,https://twitter.com/i/web/stuatus/139525828547...
4,"Portal de las Américas, anoche entre 10 y 10:3...",2021-05-20 05:56:58,Cali,"Bogotá, D.C., Colombia",1395257220267495426,https://twitter.com/i/web/stuatus/139525722026...


In [55]:
data.to_csv('archivotwitter.csv', index=False) 

In [56]:
import re

In [57]:
def text_clean(text):
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")] #Guarde las palabras que comienzan con un numeral
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
    return text, hashtags

In [58]:
pattern1 = '?P<pic>pic.twitter.com/[^\s]+' #Patrones para los links, buscar palabras que tienen el patron
pattern2 = '?P<url>https?://[^\s]+'

In [59]:
def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")] #Guarde las palabras que comienzan con un numeral
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions

In [60]:
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions
0,Duque si hizo algo por Colombia: \n\nDuque hun...,2021-05-20 06:18:58,Cali,Colombia,1395262756799004672,https://twitter.com/i/web/stuatus/139526275679...,Duque si hizo algo por Colombia: \n\nDuque hun...,[],[#PrimeraLinea],[]
1,@GobiernoBTA @andresidarragaf Que horror! No s...,2021-05-20 06:08:06,Cali,"Bogotá, D.C., Colombia",1395260023643705348,https://twitter.com/i/web/stuatus/139526002364...,Que horror! No se cansan de mentir? Lo vimos...,[],[],"[@GobiernoBTA, @andresidarragaf]"
2,Hoy en la multitudinaria movilización en Cali ...,2021-05-20 06:02:42,Cali,"Cali, Valle del Cauca",1395258664051691522,https://twitter.com/i/web/stuatus/139525866405...,Hoy en la multitudinaria movilización en Cali ...,[],[],[]
3,@VickyDavilaH @AlvaroUribeVel @IvanDuque esto ...,2021-05-20 06:01:12,Cali,"Pradera, Colombia",1395258285477990400,https://twitter.com/i/web/stuatus/139525828547...,esto pasa cuando publicas los asesinatos pe...,[https://t.co/DtC9Nv1654],[],"[@VickyDavilaH, @AlvaroUribeVel, @IvanDuque]"
4,"Portal de las Américas, anoche entre 10 y 10:3...",2021-05-20 05:56:58,Cali,"Bogotá, D.C., Colombia",1395257220267495426,https://twitter.com/i/web/stuatus/139525722026...,"Portal de las Américas, anoche entre 10 y 10:3...",[https://t.co/vSqmzddwtq],[],[]
...,...,...,...,...,...,...,...,...,...,...
495,Lo único que demuestra es que nuestros militar...,2021-05-18 17:55:16,Cali,Colombia,1394713210356436993,https://twitter.com/i/web/stuatus/139471321035...,Lo único que demuestra es que nuestros militar...,[https://t.co/KUnIHAzaTM],[],[]
496,"@elespectador Que es fake, al igual que los 50...",2021-05-18 17:52:32,Cali,Colombia,1394712521085530112,https://twitter.com/i/web/stuatus/139471252108...,"Que es fake, al igual que los 50 muertos, des...",[],[],[@elespectador]
497,La policía va a comprar 720 Biblias en cuero p...,2021-05-18 17:50:56,Cali,Colombia,1394712117845110788,https://twitter.com/i/web/stuatus/139471211784...,La policía va a comprar 720 Biblias en cuero p...,[],[],[]
498,La Policía y el ESMAD dificultan que la movili...,2021-05-18 17:50:01,Cali,Colombia,1394711890627080193,https://twitter.com/i/web/stuatus/139471189062...,La Policía y el ESMAD dificultan que la movili...,[https://t.co/pi5TCFt6Uy],[#ParoNacional18M],[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [61]:

import regex
import emoji

In [62]:
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list

In [63]:
emoji.UNICODE_EMOJI['en']

{'🥇': ':1st_place_medal:',
 '🥈': ':2nd_place_medal:',
 '🥉': ':3rd_place_medal:',
 '🆎': ':AB_button_(blood_type):',
 '🏧': ':ATM_sign:',
 '🅰': ':A_button_(blood_type):',
 '🇦🇫': ':Afghanistan:',
 '🇦🇱': ':Albania:',
 '🇩🇿': ':Algeria:',
 '🇦🇸': ':American_Samoa:',
 '🇦🇩': ':Andorra:',
 '🇦🇴': ':Angola:',
 '🇦🇮': ':Anguilla:',
 '🇦🇶': ':Antarctica:',
 '🇦🇬': ':Antigua_&_Barbuda:',
 '♒': ':Aquarius:',
 '🇦🇷': ':Argentina:',
 '♈': ':Aries:',
 '🇦🇲': ':Armenia:',
 '🇦🇼': ':Aruba:',
 '🇦🇨': ':Ascension_Island:',
 '🇦🇺': ':Australia:',
 '🇦🇹': ':Austria:',
 '🇦🇿': ':Azerbaijan:',
 '🔙': ':BACK_arrow:',
 '🅱': ':B_button_(blood_type):',
 '🇧🇸': ':Bahamas:',
 '🇧🇭': ':Bahrain:',
 '🇧🇩': ':Bangladesh:',
 '🇧🇧': ':Barbados:',
 '🇧🇾': ':Belarus:',
 '🇧🇪': ':Belgium:',
 '🇧🇿': ':Belize:',
 '🇧🇯': ':Benin:',
 '🇧🇲': ':Bermuda:',
 '🇧🇹': ':Bhutan:',
 '🇧🇴': ':Bolivia:',
 '🇧🇦': ':Bosnia_&_Herzegovina:',
 '🇧🇼': ':Botswana:',
 '🇧🇻': ':Bouvet_Island:',
 '🇧🇷': ':Brazil:',
 '🇮🇴': ':British_Indian_Ocean_Territory:',
 '🇻🇬': ':British_Vir

In [64]:
data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions,emojis
0,Duque si hizo algo por Colombia: \n\nDuque hun...,2021-05-20 06:18:58,Cali,Colombia,1395262756799004672,https://twitter.com/i/web/stuatus/139526275679...,Duque si hizo algo por Colombia: \n\nDuque hun...,[],[#PrimeraLinea],[],[]
1,@GobiernoBTA @andresidarragaf Que horror! No s...,2021-05-20 06:08:06,Cali,"Bogotá, D.C., Colombia",1395260023643705348,https://twitter.com/i/web/stuatus/139526002364...,Que horror! No se cansan de mentir? Lo vimos...,[],[],"[@GobiernoBTA, @andresidarragaf]",[]
2,Hoy en la multitudinaria movilización en Cali ...,2021-05-20 06:02:42,Cali,"Cali, Valle del Cauca",1395258664051691522,https://twitter.com/i/web/stuatus/139525866405...,Hoy en la multitudinaria movilización en Cali ...,[],[],[],[]
3,@VickyDavilaH @AlvaroUribeVel @IvanDuque esto ...,2021-05-20 06:01:12,Cali,"Pradera, Colombia",1395258285477990400,https://twitter.com/i/web/stuatus/139525828547...,esto pasa cuando publicas los asesinatos pe...,[https://t.co/DtC9Nv1654],[],"[@VickyDavilaH, @AlvaroUribeVel, @IvanDuque]",[]
4,"Portal de las Américas, anoche entre 10 y 10:3...",2021-05-20 05:56:58,Cali,"Bogotá, D.C., Colombia",1395257220267495426,https://twitter.com/i/web/stuatus/139525722026...,"Portal de las Américas, anoche entre 10 y 10:3...",[https://t.co/vSqmzddwtq],[],[],[]


In [67]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [68]:
import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [65]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
20,🤣,15
29,😂,12
27,✅,9
4,🚨,8
19,😡,7


In [69]:
trace = go.Bar(x=temp_emojis.emoji.values[:15],
               y=temp_emojis.cnt.values[:15])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)